# Projekt 1 - Ranking scenariuszy zarządzania odpadami radioaktywnymi
## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]

## Opis
Zadaniem projektu jest stworzenie rankingu różnych scenariuszy zarządzania odpadami radioaktywnymi
z wykorzystaniem odpornej regresji porządkowej. Będziemy bazować na zbiorze danych opisanym w pracy
[Nuclear waste management: an application of the multicriteria PROMETHEE methods](./paper.pdf)

Wyobraźmy sobie, że jesteśmy instytucją zajmującą się utylizacją odpadów radioaktywnych. Naszym
zadaniem jest zaproponowanie modelu utylizacji takich odpadów dla 30-letniego programu nuklearnego.
Wybrano 3 miejsca na potencjalne składowisko, 3 scenariusze czasowe oraz 3 modele finansowania. Ze
względu na to, że ostatecznie klient może się nie zgodzić na optymalną dla nas strategię, dlatego chcielibyśmy uzyskać pełne rankingi wszystkich opcji. Zespół ekspertów ocenił wszystkie możliwe scenariusze przez nas rozważane (dane znajdują się w pliku [Nuclear waste management.csv](./resources/nuclear-waste.csv))

## Informacja preferencyjna

Wybrane i oceniane pary wariantów znajdują się w pliku [preferential-information.csv](./resources/preferential-information.csv).
Grupy, do których należą autorzy to 1 i 4, które definiują preferencję.

Rozważane preferencje:
- Wydajność produkcji elektrycznej wzrośnie.
- Ostrożny inwestor, który nie chce ryzykować.


## Zadanie 1 — UTA — Ocena wariantów
### Informacja o informacji preferencyjnej.
Preferencje zostały na podstawie przynależności do grupy 1 i 4.
Te grupy definiują preferencję:
- Wydajność produkcji elektrycznej wzrośnie.
- Ostrożny inwestor, który nie chce ryzykować.

W zadaniu były dostępne 4 kryteria oceny (wszystkie są typu koszt):
- C1 — Finansowe koszty całkowite (The total financial cost)
- C2 — Koszty finansowe przez aktualnego konsumenta (The financial cost incurred by the current consumer)
- C3 — Koszty wspierane przez przyszłego konsumenta (The financial cost supported by the future consumer)
- C4 — Ryzyko kosztów finansowych (Financial risk due to overcosts)

Co zostało zinterpretowane jako minimalizacja kryterium 4 i przyzwolenie na zaniedbania kryterium 3.
Minimalizacja czwartego kryterium (C4: ryzyko kosztów finansowych) jest zgodna z preferencją ostrożnego inwestora, który nie pragnie podejmować nadmiernego ryzyka. Tymczasem zaniedbywanie trzeciego kryterium (C3: koszty wspierane przez przyszłych konsumentów) sugeruje pomijanie kosztów przyszłościowych z racji optymistycznej prognozy wzrostu wydajności w produkcji energii elektrycznej. W rezultacie modelowanie ważności kryteriów można przedstawić jako C4 >= C1 ~ C2 >= C3.

Tę hierarchię można zinterpretować następująco: pierwszym priorytetem jest minimalizacja ryzyka kosztów finansowych, aby zaspokoić potrzeby ostrożnego inwestora. Drugą preferencją są koszty całkowite, które są równoważne z trzecim priorytetem, czyli kosztami ponoszonymi przez obecnych konsumentów. W końcu najmniej ważne są koszty wspierane przez przyszłych konsumentów, ze względu na optymistyczne prognozy wzrostu wydajności w sektorze produkcji energii elektrycznej.

### Ocena wybranych par wariantów
Przedstawione niżej 5 par wariantów wchodzące do porównań zostały wybrane na podstawie informacji preferencyjnej (5,12,33) oraz predefiniowane (24,25) według indeksów autorów.

| Para | Indeks | C1   | C2   | C3   | C4   |   Preferencja   |
|:----:|:------:|------|------|------|------|:---------------:|
| 05-1 |   15   | 0.87 | 0.03 | 1.00 | 0.61 |   Preferowany   |
| 05-2 |   26   | 0.71 | 0.25 | 0.88 | 0.67 |        -        |
| 12-1 |   01   | 0.60 | 0.93 | 0.00 | 0.73 |        -        |
| 12-2 |   05   | 0.62 | 0.40 | 0.56 | 0.50 |   Preferowany   |
| 24-1 |   06   | 0.78 | 0.27 | 0.71 | 0.50 | Nierozróżnialne |
| 24-2 |   08   | 0.64 | 0.44 | 0.54 | 0.54 | Nierozróżnialne |
| 25-1 |   07   | 0.40 | 0.90 | 0.00 | 0.82 |        -        |
| 25-2 |   21   | 0.83 | 0.25 | 0.80 | 0.65 |   Preferowany   |
| 33-1 |   03   | 1.00 | 0.45 | 0.57 | 0.50 |   Preferowany   |
| 33-2 |   27   | 0.80 | 0.06 | 1.00 | 0.67 |        -        |

**Ocena par wariantów**:
- [Para 05] (15,26) — wariant 15 jest lepszy od wariantu 26 ze względu na niższą wartość przyszłego ryzyka.
- [Para 12] (01,05) — Wariant 05 jest zdecydowanie lepszy od wariantu 01 ze względu na ryzyko mimo potencjalnego kosztu przyszłego konsumenta.
- [Para 24] (06,08) — Warianty zdają się nierozróżnialne ze względu na niewielkie różnice w wartościach kryteriów.
- [Para 25] (07,21) — Wariant 21 jest lepszy od wariantu 07 ze względu na niższe ryzyko mimo drastycznego wzrostu przyszłych kosztów.
- [Para 33] (03,27) — Wariant 03 jest lepszy od wariantu 27 ze względu na niższe ryzyko.

### Problem programowania matematycznego liniowego

### Opis
Do odnalezienia rankingu został zdefiniowany problem programowania matematycznego polegający na rozwiązaniu problemu regresji porządkowej.

Problem regresji porządkowej poszukuje takiej funkcji, która umożliwia na odczytanie wartości całkowitej, dla każdego z wariantów.

Sformułowanie problemu matematycznego uwzględnia trzy typy ograniczeń:
- Odtworzenie rankingu referencyjnego, poprzez przemianowanie na sumę użyteczności cząstkowych odpowiadającym wariantom.
- Monotoniczność wartości ( w przypadku naszych kosztów jest to funkcja nierosnąca).
- Normalizacja wartości do przedziału [0,1].

In [4]:
from pulp import *
from itertools import pairwise

def flattened(it):
  return [element for iterable in it for element in iterable]
def exhaust(it):
  for _ in it: pass

In [37]:
model = LpProblem("ordinal-regression", LpMaximize)

epsilons = [LpVariable(f"e{epsilon}", cat=LpContinuous) for epsilon in range(4)]
utilities = [[LpVariable(f"u{criterion}_{i:03d}", 0, 1, LpContinuous)
              for i in range(0, 100 + 1)
              ] for criterion in range(4)]

# Funkcja celu
model.setObjective(sum(epsilons))

(u1, u2, u3, u4) = utilities

# # Warunek normalizacji — Typ koszt
model.addConstraint(sum(utility[-1] for utility in utilities) == 0)
model.addConstraint(sum(utility[0] for utility in utilities) == 1)

# Nieujemność
for value in flattened(utilities):
  model.addConstraint(value >= 0)

# Warunek monotoniczności — Typ koszt
for (v1, v2) in flattened(map(pairwise, utilities)):
  model.addConstraint(v1 <= v2)

model.addConstraint(u1[0] == 0.25)
model.addConstraint(u2[0] == 0.25)
model.addConstraint(u3[0] == 0.25)
model.addConstraint(u4[0] == 0.25)

# odtworzenie rankingu referencyjnego
model.addConstraint(u1[87] + u2[3] + u3[100] + u4[61] - u1[71] - u2[25] - u3[88] - u4[67] >= epsilons[0])
model.addConstraint(u1[62] + u2[40] + u3[56] + u4[50] - u1[60] - u2[93] - u3[0] - u4[73] >= epsilons[1])
model.addConstraint(u1[83] + u2[25] + u3[80] + u4[65] - u1[40] - u2[90] - u3[0] - u4[82] >= epsilons[2])
model.addConstraint(u1[100] + u2[45] + u3[57] + u4[50] - u1[80] - u2[6] - u3[100] - u4[67] >= epsilons[3])
model.addConstraint(u1[64] + u2[44] + u3[54] + u4[54] - u1[78] - u2[27] - u3[71] - u4[50] == 0)

model.solve()
print([value.varValue for value in epsilons])

[-0.5, -0.25, -0.5, -0.75]


### Ranking wariantów przy użyciu metody UTA

### Ocena spójności otrzymanego rankingu z informacją preferencyjną dla kilku wybranych niereferencyjnych

## Zadanie 2 — UTA GMS — Analiza koniecznych relacji

## Zadanie 3 — Konstrukcja reprezentatywnej funkcji użyteczności